# 🚀 YT-Whisper-Scribe — Configuration Colab Simplifiée

Configuration ultra-simple avec intégration GitHub directe et gestion sécurisée des cookies.

## ✅ Prérequis
1. **Activez le GPU** : `Runtime` → `Change runtime type` → `Hardware accelerator: GPU`
2. **Configuration automatique** : Exécutez la première cellule pour tout installer
3. **Cookies YouTube (optionnel)** : Suivez les instructions pour les vidéos restreintes

In [ ]:
# 🚀 CONFIGURATION AUTOMATIQUE - TOUT EN UNE CELLULE
# Cette cellule configure automatiquement tout l'environnement

import subprocess
import os

print("🔥 YT-WHISPER-SCRIBE - SETUP ULTRA-RAPIDE")
print("=" * 50)

# Téléchargement et exécution du setup avancé
if not os.path.exists('/content/colab_setup.py'):
    print("📥 Téléchargement du configurateur...")
    subprocess.run([
        'wget', '-q', 
        'https://raw.githubusercontent.com/Jojo4911/yt-whisper-scribe/main/examples/colab_setup.py',
        '-O', '/content/colab_setup.py'
    ], check=True)

# Import et exécution
exec(open('/content/colab_setup.py').read())

print("\n🎉 CONFIGURATION TERMINÉE!")
print("📖 Utilisez maintenant: transcribe_video('URL_YOUTUBE')")

## 🎬 Transcription Simple

Utilisez cette cellule pour transcription standard :

In [ ]:
# 🎬 TRANSCRIPTION SIMPLE
# Remplacez l'URL par celle de votre vidéo YouTube

url = "https://youtube.com/watch?v=VOTRE_VIDEO_ID"

# Transcription avec paramètres par défaut (SRT, modèle turbo, glossaire SWOOD)
fichier_genere = transcribe_video(url)

if fichier_genere:
    print(f"✅ Transcription terminée: {fichier_genere}")
else:
    print("❌ Erreur de transcription")

## 🔧 Transcription Avancée

Options personnalisées pour cas spécifiques :

In [ ]:
# 🔧 TRANSCRIPTION AVANCÉE
# Personnalisez selon vos besoins

url = "https://youtube.com/watch?v=VOTRE_VIDEO_ID"

fichier_genere = transcribe_video(
    url=url,
    model="medium",              # tiny, base, small, medium, large, turbo
    output_format="srt",        # srt ou txt
    language="en",               # en, fr, auto
    use_swood_glossary=True,     # Active les corrections SWOOD
    device="auto",               # auto, cuda, cpu
    verbose=True                 # Affichage détaillé
)

print(f"📁 Résultat: {fichier_genere}")

## 🍪 Gestion des Cookies YouTube

Pour les vidéos privées/restreintes, configurez vos cookies :

In [ ]:
# 🍪 MÉTHODE 1: NETTOYAGE INTERACTIF
# 1. Uploadez votre fichier cookies.txt via l'interface Colab (icône dossier à gauche)
# 2. Exécutez cette cellule pour le nettoyer automatiquement

clean_cookies_interactive()

In [ ]:
# 🍪 MÉTHODE 2: GOOGLE DRIVE (PERSISTANT)
# Sauvegarde sécurisée dans votre Google Drive

from google.colab import drive
drive.mount('/content/drive')

print("📁 Dossier privé créé: /content/drive/MyDrive/yt-whisper-private/")
print("💡 Copiez votre cookies_youtube.txt nettoyé dans ce dossier")
print("🔄 Puis relancez setup_colab_environment() pour détecter les cookies")

# Vérification manuelle
import os
private_path = "/content/drive/MyDrive/yt-whisper-private/cookies_youtube.txt"
if os.path.exists(private_path):
    print(f"✅ Cookies détectés: {private_path}")
    os.environ['YT_COOKIES_FILE'] = private_path
else:
    print("⚠️  Cookies non trouvés dans le dossier privé")

## 🎯 Démonstration Interactive

Test rapide avec entrée d'URL interactive :

In [ ]:
# 🎯 DÉMONSTRATION INTERACTIVE
# Mode interactif pour tester rapidement

demo_transcribe()

## 🔧 Dépannage et Utilitaires

In [ ]:
# 🔍 INFORMATIONS SUR LE PROJET
show_project_info()

In [ ]:
# 🔄 RÉINSTALLATION COMPLÈTE (si problème)
# Utiliser seulement en cas de problème

setup_colab_environment(force_reinstall=True)

In [ ]:
# 📁 LISTE DES FICHIERS GÉNÉRÉS
# Voir tous vos fichiers de transcription

import os
from pathlib import Path

data_dir = Path("/content/yt-whisper-scribe/data")
if data_dir.exists():
    files = list(data_dir.glob("*"))
    print(f"📁 {len(files)} fichier(s) dans data/:")
    for f in sorted(files, key=os.path.getmtime, reverse=True):
        size = f.stat().st_size / 1024  # KB
        print(f"  📄 {f.name} ({size:.1f} KB)")
else:
    print("📁 Dossier data/ non trouvé")

## 📖 Guide d'Utilisation Rapide

### 🚀 Démarrage Ultra-Rapide
1. Exécutez la **première cellule** pour configuration automatique
2. Modifiez l'URL dans la cellule **Transcription Simple**
3. Exécutez et attendez le résultat!

### 🍪 Pour les Vidéos Restreintes
1. Exportez vos cookies depuis votre navigateur (extension "Get cookies.txt")
2. Utilisez `clean_cookies_interactive()` pour les nettoyer
3. Sauvegardez dans Google Drive pour persistance

### 🎛️ Options Principales
- **model**: `"tiny"` (rapide) → `"turbo"` (équilibré) → `"large"` (précis)
- **language**: `"en"`, `"fr"`, `"auto"` (détection automatique)
- **output_format**: `"srt"` (sous-titres) ou `"txt"` (texte brut)
- **use_swood_glossary**: `True` pour corrections métier automatiques

### ⚡ Fonctions Utiles
```python
# Configuration initiale
setup_colab_environment()

# Transcription simple
transcribe_video("https://youtube.com/watch?v=VIDEO_ID")

# Nettoyage cookies
clean_cookies_interactive()

# Test interactif
demo_transcribe()
```

### 🔧 Résolution de Problèmes
- **GPU non détecté**: Runtime → Change runtime type → GPU
- **Vidéo inaccessible**: Configurez vos cookies YouTube
- **Mémoire insuffisante**: Utilisez `model="base"` ou `device="cpu"`
- **Erreur de clonage**: Vérifiez votre connexion réseau